### 12.03 Implement and ISBN cache

The International Standard Book Number (ISBN) is a unique commercial book identifier.  It is a string of length 10.  The first 9 characters are digits; the last character is a check character.  The check character is the sum of the first 9 digits, mod 11, with 10 represented by 'X'.  

Create a cache for looking up prices of books identified by their ISBN.  You implement lookup, insert, and remove methods.  Use the Least Recently Used (LRU) policy for cache eviction.  If an ISBN is already present, insert should not change the price, but it should update that entry to be the most recently used entry.

***Hint***: Amortize the cost of deletion.  Alternatively, use an auxiliary data structure.